In [1]:
### basic package for data science project
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import helpers

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set()

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [3]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [95]:
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [400]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, auc, precision_recall_curve, roc_curve, average_precision_score

### Helpers

In [408]:
def reporting(ensem_preds, targets):
    best_th = 0
    best_score = 0

    for th in np.arange(0.0, 0.6, 0.01):
        pred = (ensem_preds > th).astype(int)
        score = f1_score(targets, pred)
        if score > best_score:
            best_th = th
            best_score = score

    print(f"\nAUC score: {roc_auc_score(targets, ensem_preds):12.4f}")
    print(f"Best threshold {best_th:12.4f}")
    preds = (ensem_preds > best_th).astype(int)
#     print(classification_report(targets, preds, digits=3))


    print(f"Report with best threshold : \n")
    cm1 = confusion_matrix(targets, preds)
    print('\nConfusion Matrix : \n', cm1)
    total1=sum(sum(cm1))

    print('\n=============')
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print (f'Accuracy    : {accuracy1:12.4f}')

    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[1,0])
    print(f'Sensitivity : {sensitivity1:12.4f}')

    specificity1 = cm1[1,1]/(cm1[0,1]+cm1[1,1])
    print(f'Specificity : {specificity1:12.4f}')

1. Remove columns with missing value more than 75% 
2. Remove col 'x39' and 'x99' since those two columns don't bring additional information 
3. Impute NAN for missing values in numerical methods 
4. We can use Catboost or Imputing categorical features is to replace missing values with the most common class. 


Alternative (if time permitted):
    - Check each features to see if there exists outliers.

## A. Feature Engineering

In [137]:
train_data = pd.read_csv('data/exercise_40_train.csv')
test_data = pd.read_csv('data/exercise_40_test.csv')

In [227]:
most_missing_cols = list(set(train_data.columns[train_data.isnull().mean() > 0.75]))
remove_cat_col = ['x39', 'x99']
remove_cols = most_missing_cols + remove_cat_col

In [228]:
def create_dummy_df(df, cat_cols, dummy_na):
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1),\
                            pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

def clean_catergorical_ft(df):
    
    cat_df = df.select_dtypes(include=['object'])
    
    #### clean col x7 and x19
    cat_df['x7'] = cat_df['x7'].str.replace('%','')
    cat_df["x19"] = cat_df['x19'].str.replace('$','')
    cat_df = cat_df.astype({'x7': 'float', 'x19': 'float'})

    #### clean x3 col
    cat_df['x3'] = cat_df.apply(lambda x: helpers.clean_day_x3_col(x['x3']), axis = 1)

    ### convert x7 and x19 back to numerical fts 
    df['x7'] =  cat_df['x7']
    df['x19'] = cat_df["x19"]

    #### categorical cols 
    cat_df.drop(['x7','x19'], axis = 1, inplace = True)
    cat_cols = cat_df.columns
    
    return cat_cols, cat_df, df
    

In [335]:
def ft_engineering_test(df, remove_cols, use_catboost = True):
    """
    This function is used to clean the test dataset and extract features from it
    """
    df = df.loc[:, (~df.columns.isin(remove_cols))]
    print(df.shape)
    #### clean categorical features
    cat_cols, cat_df, df = clean_catergorical_ft(df)
    
    #### clean numerical features 
    numerical_df = df.loc[:, (~df.columns.isin(cat_cols))]
    
    
    ### if not using catboost model, then imputing categorical features 
    ### by replacing missing values with the most common class
    if use_catboost == False:
        
        num_df_trasformed = numerical_df.fillna(0)
        
        ### replace missing value with most common value
        for col in cat_cols:
            max_freq = cat_df[col].value_counts().index[0]
            cat_df[col][pd.isna(cat_df[col])] = max_freq
    
        with open('one_hot_encoder.pickle', 'rb') as f:
            one_hot_encoder = pickle.load(f)
        cat_df_transformed = pd.DataFrame(one_hot_encoder.transform(cat_df).toarray())

    else:
        cat_df_transformed = cat_df.fillna('NaN')
        num_df_trasformed = numerical_df.fillna('NaN')
       
    cleaned_data = pd.concat([num_df_trasformed, cat_df_transformed.reset_index(drop = True)], axis=1)

    return cleaned_data


In [287]:
def ft_engineering_train(df, remove_cols, use_catboost = True):
    df = df.loc[:, (~df.columns.isin(remove_cols))]
 
    #### clean categorical features
    cat_cols, cat_df, df = clean_catergorical_ft(df)
    
    #### clean numerical features 
    numerical_df = df.loc[:, (~df.columns.isin(cat_cols))]
            
    ### if not using catboost model, then imputing categorical features 
    ### by replacing missing values with the most common class
    if use_catboost == False:
        
        num_df_trasformed = numerical_df.fillna(0)
        
        ### replace missing value with most common value
        for col in cat_cols:
            max_freq = cat_df[col].value_counts().index[0] 
            cat_df[col][pd.isna(cat_df[col])] = max_freq
            
        one_hot_encoder = OneHotEncoder()
        cat_df_transformed = pd.DataFrame(one_hot_encoder.fit_transform(cat_df).toarray())
        with open('one_hot_encoder.pickle', 'wb') as f:
            pickle.dump(one_hot_encoder, f, pickle.HIGHEST_PROTOCOL)
      
    else:
        cat_df_transformed = cat_df.fillna('NaN')
        num_df_trasformed = numerical_df.fillna('NaN')
    
    cleaned_data = pd.concat([num_df_trasformed, cat_df_transformed.reset_index(drop = True)], axis=1)

    cleaned_data["y"] = df['y'].reset_index(drop = True)

    return cleaned_data


## B. Using catboost 

In [289]:
data_cb = ft_engineering_train(train_data, remove_cols, use_catboost = True)


### 1. Hyper-parameter tunning

In [369]:
# target
y = data_cb['y']

# features

X = data_cb.loc[:, ~data_cb.columns.isin(['y'])]

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier(class_weights={0:1, 1:6})

# Creating the hyperparameter grid
param_dist = { "learning_rate":[0.01, 0.05, 0.1, 0.5],
               "max_depth": [3, 5, 6, 7], 
              "n_estimators": [500, 1000, 1500]
             }
               
#Instantiate RandomSearchCV object
rscv = RandomizedSearchCV(cbc, param_dist, scoring='accuracy', cv =5)

#Fit the model
categorical_features_indices = [87, 88, 89, 90, 91, 92, 93, 94]
rscv.fit(X,y,cat_features=categorical_features_indices, plot = True)

# Print the tuned parameters and score
print(rscv.best_params_)
print(rscv.best_score_)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6850440	total: 87.8ms	remaining: 1m 27s
1:	learn: 0.6761772	total: 124ms	remaining: 1m 2s
2:	learn: 0.6685624	total: 174ms	remaining: 57.8s
3:	learn: 0.6610548	total: 224ms	remaining: 55.9s
4:	learn: 0.6541901	total: 309ms	remaining: 1m 1s
5:	learn: 0.6480101	total: 350ms	remaining: 58.1s
6:	learn: 0.6430918	total: 390ms	remaining: 55.3s
7:	learn: 0.6384820	total: 455ms	remaining: 56.4s
8:	learn: 0.6337756	total: 544ms	remaining: 59.9s
9:	learn: 0.6304260	total: 601ms	remaining: 59.5s
10:	learn: 0.6271164	total: 640ms	remaining: 57.6s
11:	learn: 0.6230793	total: 696ms	remaining: 57.3s
12:	learn: 0.6189381	total: 766ms	remaining: 58.1s
13:	learn: 0.6150025	total: 813ms	remaining: 57.3s
14:	learn: 0.6121295	total: 854ms	remaining: 56.1s
15:	learn: 0.6097701	total: 901ms	remaining: 55.4s
16:	learn: 0.6066388	total: 992ms	remaining: 57.4s
17:	learn: 0.6044623	total: 1.03s	remaining: 56.5s
18:	learn: 0.6019035	total: 1.07s	remaining: 55.6s
19:	learn: 0.5999167	total: 1.13s	remai

163:	learn: 0.5261582	total: 9.18s	remaining: 46.8s
164:	learn: 0.5258868	total: 9.24s	remaining: 46.8s
165:	learn: 0.5256613	total: 9.28s	remaining: 46.6s
166:	learn: 0.5255154	total: 9.32s	remaining: 46.5s
167:	learn: 0.5253688	total: 9.37s	remaining: 46.4s
168:	learn: 0.5252562	total: 9.48s	remaining: 46.6s
169:	learn: 0.5251019	total: 9.52s	remaining: 46.5s
170:	learn: 0.5248936	total: 9.56s	remaining: 46.4s
171:	learn: 0.5247413	total: 9.62s	remaining: 46.3s
172:	learn: 0.5244927	total: 9.71s	remaining: 46.4s
173:	learn: 0.5242893	total: 9.75s	remaining: 46.3s
174:	learn: 0.5240556	total: 9.79s	remaining: 46.2s
175:	learn: 0.5239345	total: 9.84s	remaining: 46.1s
176:	learn: 0.5237975	total: 9.91s	remaining: 46.1s
177:	learn: 0.5237132	total: 9.97s	remaining: 46s
178:	learn: 0.5234780	total: 10s	remaining: 46s
179:	learn: 0.5232382	total: 10.1s	remaining: 45.9s
180:	learn: 0.5229968	total: 10.2s	remaining: 46s
181:	learn: 0.5227994	total: 10.2s	remaining: 45.9s
182:	learn: 0.522672

322:	learn: 0.4886543	total: 17.8s	remaining: 37.2s
323:	learn: 0.4884065	total: 17.8s	remaining: 37.1s
324:	learn: 0.4881470	total: 17.9s	remaining: 37.1s
325:	learn: 0.4879407	total: 17.9s	remaining: 37s
326:	learn: 0.4876848	total: 18s	remaining: 37s
327:	learn: 0.4873597	total: 18s	remaining: 36.9s
328:	learn: 0.4871568	total: 18.1s	remaining: 36.9s
329:	learn: 0.4869192	total: 18.1s	remaining: 36.8s
330:	learn: 0.4866835	total: 18.2s	remaining: 36.7s
331:	learn: 0.4864307	total: 18.2s	remaining: 36.7s
332:	learn: 0.4862121	total: 18.3s	remaining: 36.7s
333:	learn: 0.4859122	total: 18.4s	remaining: 36.6s
334:	learn: 0.4856409	total: 18.4s	remaining: 36.6s
335:	learn: 0.4854160	total: 18.5s	remaining: 36.5s
336:	learn: 0.4852562	total: 18.6s	remaining: 36.6s
337:	learn: 0.4849948	total: 18.6s	remaining: 36.5s
338:	learn: 0.4847593	total: 18.7s	remaining: 36.4s
339:	learn: 0.4845114	total: 18.7s	remaining: 36.4s
340:	learn: 0.4842984	total: 18.8s	remaining: 36.4s
341:	learn: 0.484067

482:	learn: 0.4535511	total: 26.4s	remaining: 28.3s
483:	learn: 0.4533363	total: 26.5s	remaining: 28.2s
484:	learn: 0.4531116	total: 26.6s	remaining: 28.2s
485:	learn: 0.4529037	total: 26.6s	remaining: 28.1s
486:	learn: 0.4526878	total: 26.6s	remaining: 28.1s
487:	learn: 0.4524700	total: 26.7s	remaining: 28s
488:	learn: 0.4522770	total: 26.8s	remaining: 28s
489:	learn: 0.4521451	total: 26.8s	remaining: 27.9s
490:	learn: 0.4519080	total: 26.8s	remaining: 27.8s
491:	learn: 0.4516879	total: 26.9s	remaining: 27.8s
492:	learn: 0.4514970	total: 27s	remaining: 27.8s
493:	learn: 0.4512928	total: 27.1s	remaining: 27.7s
494:	learn: 0.4511008	total: 27.1s	remaining: 27.7s
495:	learn: 0.4508912	total: 27.2s	remaining: 27.6s
496:	learn: 0.4506326	total: 27.2s	remaining: 27.5s
497:	learn: 0.4503722	total: 27.3s	remaining: 27.5s
498:	learn: 0.4501606	total: 27.3s	remaining: 27.4s
499:	learn: 0.4498996	total: 27.3s	remaining: 27.3s
500:	learn: 0.4497888	total: 27.4s	remaining: 27.3s
501:	learn: 0.4495

...

796:	learn: 0.0660464	total: 48s	remaining: 42.3s
797:	learn: 0.0658363	total: 48s	remaining: 42.2s
798:	learn: 0.0656800	total: 48.1s	remaining: 42.2s
799:	learn: 0.0655257	total: 48.1s	remaining: 42.1s
800:	learn: 0.0654330	total: 48.2s	remaining: 42.1s
801:	learn: 0.0652609	total: 48.3s	remaining: 42s
802:	learn: 0.0650916	total: 48.3s	remaining: 42s
803:	learn: 0.0649179	total: 48.4s	remaining: 41.9s
804:	learn: 0.0648108	total: 48.5s	remaining: 41.8s
805:	learn: 0.0646088	total: 48.5s	remaining: 41.8s
806:	learn: 0.0645032	total: 48.6s	remaining: 41.7s
807:	learn: 0.0643503	total: 48.6s	remaining: 41.6s
808:	learn: 0.0642881	total: 48.7s	remaining: 41.6s
809:	learn: 0.0641088	total: 48.7s	remaining: 41.5s
810:	learn: 0.0639680	total: 48.8s	remaining: 41.5s
811:	learn: 0.0638706	total: 48.9s	remaining: 41.4s
812:	learn: 0.0637433	total: 48.9s	remaining: 41.3s
813:	learn: 0.0635325	total: 49s	remaining: 41.3s
814:	learn: 0.0633348	total: 49s	remaining: 41.2s
815:	learn: 0.0632153	to

956:	learn: 0.0457066	total: 57.5s	remaining: 32.6s
957:	learn: 0.0456568	total: 57.6s	remaining: 32.6s
958:	learn: 0.0455506	total: 57.6s	remaining: 32.5s
959:	learn: 0.0454723	total: 57.7s	remaining: 32.4s
960:	learn: 0.0453976	total: 57.8s	remaining: 32.4s
961:	learn: 0.0452843	total: 57.8s	remaining: 32.3s
962:	learn: 0.0451939	total: 57.9s	remaining: 32.3s
963:	learn: 0.0450870	total: 57.9s	remaining: 32.2s
964:	learn: 0.0449631	total: 58s	remaining: 32.2s
965:	learn: 0.0448572	total: 58s	remaining: 32.1s
966:	learn: 0.0447382	total: 58.1s	remaining: 32s
967:	learn: 0.0446311	total: 58.2s	remaining: 32s
968:	learn: 0.0444742	total: 58.2s	remaining: 31.9s
969:	learn: 0.0443366	total: 58.3s	remaining: 31.8s
970:	learn: 0.0442369	total: 58.3s	remaining: 31.8s
971:	learn: 0.0440953	total: 58.4s	remaining: 31.7s
972:	learn: 0.0440116	total: 58.5s	remaining: 31.7s
973:	learn: 0.0439082	total: 58.5s	remaining: 31.6s
974:	learn: 0.0438332	total: 58.6s	remaining: 31.5s
975:	learn: 0.043735

1116:	learn: 0.0328807	total: 1m 7s	remaining: 23.1s
1117:	learn: 0.0328186	total: 1m 7s	remaining: 23s
1118:	learn: 0.0327604	total: 1m 7s	remaining: 22.9s
1119:	learn: 0.0327048	total: 1m 7s	remaining: 22.9s
1120:	learn: 0.0326657	total: 1m 7s	remaining: 22.9s
1121:	learn: 0.0325846	total: 1m 7s	remaining: 22.8s
1122:	learn: 0.0325107	total: 1m 7s	remaining: 22.7s
1123:	learn: 0.0324276	total: 1m 7s	remaining: 22.7s
1124:	learn: 0.0323822	total: 1m 7s	remaining: 22.6s
1125:	learn: 0.0323369	total: 1m 7s	remaining: 22.5s
1126:	learn: 0.0322416	total: 1m 7s	remaining: 22.5s
1127:	learn: 0.0321700	total: 1m 8s	remaining: 22.4s
1128:	learn: 0.0320723	total: 1m 8s	remaining: 22.4s
1129:	learn: 0.0319860	total: 1m 8s	remaining: 22.3s
1130:	learn: 0.0318826	total: 1m 8s	remaining: 22.3s
1131:	learn: 0.0317983	total: 1m 8s	remaining: 22.2s
1132:	learn: 0.0317523	total: 1m 8s	remaining: 22.1s
1133:	learn: 0.0316808	total: 1m 8s	remaining: 22.1s
1134:	learn: 0.0315993	total: 1m 8s	remaining: 2

1272:	learn: 0.0238332	total: 1m 17s	remaining: 13.7s
1273:	learn: 0.0237968	total: 1m 17s	remaining: 13.7s
1274:	learn: 0.0237467	total: 1m 17s	remaining: 13.6s
1275:	learn: 0.0236965	total: 1m 17s	remaining: 13.6s
1276:	learn: 0.0236629	total: 1m 17s	remaining: 13.5s
1277:	learn: 0.0236225	total: 1m 17s	remaining: 13.4s
1278:	learn: 0.0235790	total: 1m 17s	remaining: 13.4s
1279:	learn: 0.0235180	total: 1m 17s	remaining: 13.3s
1280:	learn: 0.0234852	total: 1m 17s	remaining: 13.3s
1281:	learn: 0.0234210	total: 1m 17s	remaining: 13.2s
1282:	learn: 0.0233624	total: 1m 17s	remaining: 13.1s
1283:	learn: 0.0233332	total: 1m 17s	remaining: 13.1s
1284:	learn: 0.0232864	total: 1m 17s	remaining: 13s
1285:	learn: 0.0232376	total: 1m 17s	remaining: 12.9s
1286:	learn: 0.0231948	total: 1m 17s	remaining: 12.9s
1287:	learn: 0.0231380	total: 1m 17s	remaining: 12.8s
1288:	learn: 0.0230906	total: 1m 18s	remaining: 12.8s
1289:	learn: 0.0230189	total: 1m 18s	remaining: 12.7s
1290:	learn: 0.0229681	total: 

1428:	learn: 0.0175912	total: 1m 26s	remaining: 4.3s
1429:	learn: 0.0175692	total: 1m 26s	remaining: 4.24s
1430:	learn: 0.0175341	total: 1m 26s	remaining: 4.17s
1431:	learn: 0.0175220	total: 1m 26s	remaining: 4.11s
1432:	learn: 0.0174886	total: 1m 26s	remaining: 4.05s
1433:	learn: 0.0174594	total: 1m 26s	remaining: 3.99s
1434:	learn: 0.0174404	total: 1m 26s	remaining: 3.93s
1435:	learn: 0.0173905	total: 1m 26s	remaining: 3.87s
1436:	learn: 0.0173712	total: 1m 26s	remaining: 3.81s
1437:	learn: 0.0173466	total: 1m 26s	remaining: 3.75s
1438:	learn: 0.0172973	total: 1m 27s	remaining: 3.69s
1439:	learn: 0.0172810	total: 1m 27s	remaining: 3.63s
1440:	learn: 0.0172482	total: 1m 27s	remaining: 3.57s
1441:	learn: 0.0172076	total: 1m 27s	remaining: 3.51s
1442:	learn: 0.0172043	total: 1m 27s	remaining: 3.45s
1443:	learn: 0.0171724	total: 1m 27s	remaining: 3.39s
1444:	learn: 0.0171426	total: 1m 27s	remaining: 3.33s
1445:	learn: 0.0170996	total: 1m 27s	remaining: 3.27s
1446:	learn: 0.0170618	total:

### 2. Train and validate model 
- We split data normally since the way we do feature engineering already handles data leakage.
- Catboost will automatically handle missing values, so when we split the train and test from exercise_40_train.csv, no leakage occurs.

In [371]:
X_train, X_test, y_train, y_test = train_test_split(
                                    data_cb.loc[:,~data_cb.columns.isin(['y'])],
                                    data_cb['y'],
                                    test_size=0.1,
                                    random_state=0)

In [424]:
X_train, X_test, y_train, y_test = train_test_split(
                                    data_cb.loc[:,~data_cb.columns.isin(['y'])],
                                    data_cb['y'],
                                    test_size=0.1,
                                    random_state=0)

# categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
categorical_features_indices = [87, 88, 89, 90, 91, 92, 93, 94]
cbc = CatBoostClassifier(learning_rate= 0.05, max_depth= 5, n_estimators= 2500,\
                         cat_features=categorical_features_indices)
cbc.fit(X_train,y_train)

0:	learn: 0.6566153	total: 106ms	remaining: 4m 25s
1:	learn: 0.6261936	total: 150ms	remaining: 3m 7s
2:	learn: 0.6003371	total: 196ms	remaining: 2m 43s
3:	learn: 0.5784672	total: 256ms	remaining: 2m 39s
4:	learn: 0.5565555	total: 326ms	remaining: 2m 42s
5:	learn: 0.5365029	total: 373ms	remaining: 2m 34s
6:	learn: 0.5191764	total: 414ms	remaining: 2m 27s
7:	learn: 0.5030507	total: 483ms	remaining: 2m 30s
8:	learn: 0.4898925	total: 548ms	remaining: 2m 31s
9:	learn: 0.4766196	total: 596ms	remaining: 2m 28s
10:	learn: 0.4654291	total: 649ms	remaining: 2m 26s
11:	learn: 0.4548691	total: 712ms	remaining: 2m 27s
12:	learn: 0.4457719	total: 800ms	remaining: 2m 32s
13:	learn: 0.4372451	total: 846ms	remaining: 2m 30s
14:	learn: 0.4294241	total: 891ms	remaining: 2m 27s
15:	learn: 0.4234249	total: 937ms	remaining: 2m 25s
16:	learn: 0.4193746	total: 1.01s	remaining: 2m 27s
17:	learn: 0.4158252	total: 1.03s	remaining: 2m 22s
18:	learn: 0.4099162	total: 1.08s	remaining: 2m 21s
19:	learn: 0.4051474	to

159:	learn: 0.3297882	total: 9.34s	remaining: 2m 16s
160:	learn: 0.3296838	total: 9.39s	remaining: 2m 16s
161:	learn: 0.3296101	total: 9.45s	remaining: 2m 16s
162:	learn: 0.3294915	total: 9.52s	remaining: 2m 16s
163:	learn: 0.3294253	total: 9.57s	remaining: 2m 16s
164:	learn: 0.3292831	total: 9.62s	remaining: 2m 16s
165:	learn: 0.3291793	total: 9.67s	remaining: 2m 15s
166:	learn: 0.3291130	total: 9.76s	remaining: 2m 16s
167:	learn: 0.3290151	total: 9.8s	remaining: 2m 16s
168:	learn: 0.3289245	total: 9.85s	remaining: 2m 15s
169:	learn: 0.3288429	total: 9.9s	remaining: 2m 15s
170:	learn: 0.3286988	total: 9.98s	remaining: 2m 15s
171:	learn: 0.3286142	total: 10s	remaining: 2m 15s
172:	learn: 0.3285332	total: 10.1s	remaining: 2m 15s
173:	learn: 0.3284625	total: 10.1s	remaining: 2m 15s
174:	learn: 0.3283513	total: 10.2s	remaining: 2m 15s
175:	learn: 0.3282382	total: 10.3s	remaining: 2m 15s
176:	learn: 0.3281483	total: 10.3s	remaining: 2m 15s
177:	learn: 0.3280634	total: 10.4s	remaining: 2m 1

316:	learn: 0.3139968	total: 18.3s	remaining: 2m 5s
317:	learn: 0.3138646	total: 18.3s	remaining: 2m 5s
318:	learn: 0.3138119	total: 18.4s	remaining: 2m 5s
319:	learn: 0.3137492	total: 18.5s	remaining: 2m 5s
320:	learn: 0.3136621	total: 18.5s	remaining: 2m 5s
321:	learn: 0.3135580	total: 18.6s	remaining: 2m 5s
322:	learn: 0.3134369	total: 18.7s	remaining: 2m 5s
323:	learn: 0.3133193	total: 18.7s	remaining: 2m 5s
324:	learn: 0.3132582	total: 18.7s	remaining: 2m 5s
325:	learn: 0.3131649	total: 18.8s	remaining: 2m 5s
326:	learn: 0.3130495	total: 18.9s	remaining: 2m 5s
327:	learn: 0.3129381	total: 18.9s	remaining: 2m 5s
328:	learn: 0.3128206	total: 19s	remaining: 2m 5s
329:	learn: 0.3127166	total: 19s	remaining: 2m 5s
330:	learn: 0.3126490	total: 19.1s	remaining: 2m 5s
331:	learn: 0.3125379	total: 19.1s	remaining: 2m 4s
332:	learn: 0.3124482	total: 19.2s	remaining: 2m 4s
333:	learn: 0.3123561	total: 19.2s	remaining: 2m 4s
334:	learn: 0.3122245	total: 19.3s	remaining: 2m 4s
335:	learn: 0.31

476:	learn: 0.2997370	total: 27.5s	remaining: 1m 56s
477:	learn: 0.2996396	total: 27.5s	remaining: 1m 56s
478:	learn: 0.2995963	total: 27.6s	remaining: 1m 56s
479:	learn: 0.2994935	total: 27.7s	remaining: 1m 56s
480:	learn: 0.2993831	total: 27.7s	remaining: 1m 56s
481:	learn: 0.2993183	total: 27.8s	remaining: 1m 56s
482:	learn: 0.2991986	total: 27.9s	remaining: 1m 56s
483:	learn: 0.2991136	total: 27.9s	remaining: 1m 56s
484:	learn: 0.2990228	total: 28s	remaining: 1m 56s
485:	learn: 0.2989345	total: 28s	remaining: 1m 56s
486:	learn: 0.2989136	total: 28.1s	remaining: 1m 56s
487:	learn: 0.2988284	total: 28.1s	remaining: 1m 56s
488:	learn: 0.2987389	total: 28.2s	remaining: 1m 55s
489:	learn: 0.2986522	total: 28.2s	remaining: 1m 55s
490:	learn: 0.2985977	total: 28.3s	remaining: 1m 55s
491:	learn: 0.2985412	total: 28.4s	remaining: 1m 55s
492:	learn: 0.2984573	total: 28.4s	remaining: 1m 55s
493:	learn: 0.2983369	total: 28.5s	remaining: 1m 55s
494:	learn: 0.2982499	total: 28.6s	remaining: 1m 5

632:	learn: 0.2875531	total: 36.6s	remaining: 1m 48s
633:	learn: 0.2875085	total: 36.7s	remaining: 1m 47s
634:	learn: 0.2874217	total: 36.8s	remaining: 1m 48s
635:	learn: 0.2873107	total: 36.8s	remaining: 1m 47s
636:	learn: 0.2872175	total: 36.9s	remaining: 1m 47s
637:	learn: 0.2871240	total: 36.9s	remaining: 1m 47s
638:	learn: 0.2870441	total: 37s	remaining: 1m 47s
639:	learn: 0.2869927	total: 37.1s	remaining: 1m 47s
640:	learn: 0.2869287	total: 37.1s	remaining: 1m 47s
641:	learn: 0.2868615	total: 37.2s	remaining: 1m 47s
642:	learn: 0.2868109	total: 37.3s	remaining: 1m 47s
643:	learn: 0.2866909	total: 37.3s	remaining: 1m 47s
644:	learn: 0.2866171	total: 37.4s	remaining: 1m 47s
645:	learn: 0.2865016	total: 37.4s	remaining: 1m 47s
646:	learn: 0.2864204	total: 37.5s	remaining: 1m 47s
647:	learn: 0.2863086	total: 37.5s	remaining: 1m 47s
648:	learn: 0.2862196	total: 37.6s	remaining: 1m 47s
649:	learn: 0.2861549	total: 37.6s	remaining: 1m 47s
650:	learn: 0.2860539	total: 37.7s	remaining: 1m

...

1866:	learn: 0.2094820	total: 1m 48s	remaining: 36.9s
1867:	learn: 0.2094332	total: 1m 48s	remaining: 36.8s
1868:	learn: 0.2093836	total: 1m 48s	remaining: 36.8s
1869:	learn: 0.2093474	total: 1m 49s	remaining: 36.7s
1870:	learn: 0.2092940	total: 1m 49s	remaining: 36.7s
1871:	learn: 0.2092594	total: 1m 49s	remaining: 36.6s
1872:	learn: 0.2092395	total: 1m 49s	remaining: 36.5s
1873:	learn: 0.2091738	total: 1m 49s	remaining: 36.5s
1874:	learn: 0.2091097	total: 1m 49s	remaining: 36.4s
1875:	learn: 0.2090426	total: 1m 49s	remaining: 36.4s
1876:	learn: 0.2090032	total: 1m 49s	remaining: 36.3s
1877:	learn: 0.2089313	total: 1m 49s	remaining: 36.3s
1878:	learn: 0.2088908	total: 1m 49s	remaining: 36.2s
1879:	learn: 0.2088420	total: 1m 49s	remaining: 36.1s
1880:	learn: 0.2087813	total: 1m 49s	remaining: 36.1s
1881:	learn: 0.2087194	total: 1m 49s	remaining: 36s
1882:	learn: 0.2086792	total: 1m 49s	remaining: 36s
1883:	learn: 0.2086027	total: 1m 49s	remaining: 35.9s
1884:	learn: 0.2085377	total: 1m

2019:	learn: 0.2017059	total: 1m 58s	remaining: 28s
2020:	learn: 0.2016350	total: 1m 58s	remaining: 28s
2021:	learn: 0.2015703	total: 1m 58s	remaining: 27.9s
2022:	learn: 0.2015399	total: 1m 58s	remaining: 27.9s
2023:	learn: 0.2014945	total: 1m 58s	remaining: 27.8s
2024:	learn: 0.2014526	total: 1m 58s	remaining: 27.8s
2025:	learn: 0.2014047	total: 1m 58s	remaining: 27.7s
2026:	learn: 0.2013510	total: 1m 58s	remaining: 27.6s
2027:	learn: 0.2012991	total: 1m 58s	remaining: 27.6s
2028:	learn: 0.2012378	total: 1m 58s	remaining: 27.5s
2029:	learn: 0.2011858	total: 1m 58s	remaining: 27.5s
2030:	learn: 0.2011311	total: 1m 58s	remaining: 27.4s
2031:	learn: 0.2010843	total: 1m 58s	remaining: 27.4s
2032:	learn: 0.2010303	total: 1m 58s	remaining: 27.3s
2033:	learn: 0.2009709	total: 1m 58s	remaining: 27.2s
2034:	learn: 0.2009092	total: 1m 58s	remaining: 27.2s
2035:	learn: 0.2008452	total: 1m 59s	remaining: 27.1s
2036:	learn: 0.2008191	total: 1m 59s	remaining: 27.1s
2037:	learn: 0.2007877	total: 1m

2178:	learn: 0.1943625	total: 2m 7s	remaining: 18.8s
2179:	learn: 0.1943054	total: 2m 7s	remaining: 18.7s
2180:	learn: 0.1942587	total: 2m 7s	remaining: 18.7s
2181:	learn: 0.1941958	total: 2m 7s	remaining: 18.6s
2182:	learn: 0.1941723	total: 2m 7s	remaining: 18.6s
2183:	learn: 0.1941046	total: 2m 7s	remaining: 18.5s
2184:	learn: 0.1940665	total: 2m 7s	remaining: 18.4s
2185:	learn: 0.1940107	total: 2m 7s	remaining: 18.4s
2186:	learn: 0.1939561	total: 2m 8s	remaining: 18.3s
2187:	learn: 0.1939262	total: 2m 8s	remaining: 18.3s
2188:	learn: 0.1938881	total: 2m 8s	remaining: 18.2s
2189:	learn: 0.1938440	total: 2m 8s	remaining: 18.2s
2190:	learn: 0.1937844	total: 2m 8s	remaining: 18.1s
2191:	learn: 0.1937232	total: 2m 8s	remaining: 18s
2192:	learn: 0.1936848	total: 2m 8s	remaining: 18s
2193:	learn: 0.1936247	total: 2m 8s	remaining: 17.9s
2194:	learn: 0.1935849	total: 2m 8s	remaining: 17.9s
2195:	learn: 0.1935368	total: 2m 8s	remaining: 17.8s
2196:	learn: 0.1935026	total: 2m 8s	remaining: 17.

2331:	learn: 0.1872231	total: 2m 16s	remaining: 9.84s
2332:	learn: 0.1871805	total: 2m 16s	remaining: 9.78s
2333:	learn: 0.1871631	total: 2m 16s	remaining: 9.72s
2334:	learn: 0.1871367	total: 2m 16s	remaining: 9.66s
2335:	learn: 0.1870851	total: 2m 16s	remaining: 9.6s
2336:	learn: 0.1870346	total: 2m 16s	remaining: 9.54s
2337:	learn: 0.1869978	total: 2m 16s	remaining: 9.49s
2338:	learn: 0.1869600	total: 2m 16s	remaining: 9.43s
2339:	learn: 0.1868897	total: 2m 17s	remaining: 9.37s
2340:	learn: 0.1868546	total: 2m 17s	remaining: 9.31s
2341:	learn: 0.1867867	total: 2m 17s	remaining: 9.25s
2342:	learn: 0.1867508	total: 2m 17s	remaining: 9.19s
2343:	learn: 0.1867043	total: 2m 17s	remaining: 9.13s
2344:	learn: 0.1866722	total: 2m 17s	remaining: 9.07s
2345:	learn: 0.1866085	total: 2m 17s	remaining: 9.02s
2346:	learn: 0.1865536	total: 2m 17s	remaining: 8.96s
2347:	learn: 0.1864912	total: 2m 17s	remaining: 8.9s
2348:	learn: 0.1864760	total: 2m 17s	remaining: 8.84s
2349:	learn: 0.1864348	total: 

2485:	learn: 0.1802362	total: 2m 25s	remaining: 820ms
2486:	learn: 0.1802064	total: 2m 25s	remaining: 762ms
2487:	learn: 0.1801498	total: 2m 25s	remaining: 703ms
2488:	learn: 0.1801010	total: 2m 25s	remaining: 645ms
2489:	learn: 0.1800560	total: 2m 25s	remaining: 586ms
2490:	learn: 0.1799905	total: 2m 25s	remaining: 527ms
2491:	learn: 0.1799491	total: 2m 26s	remaining: 469ms
2492:	learn: 0.1799069	total: 2m 26s	remaining: 410ms
2493:	learn: 0.1798764	total: 2m 26s	remaining: 352ms
2494:	learn: 0.1798194	total: 2m 26s	remaining: 293ms
2495:	learn: 0.1797651	total: 2m 26s	remaining: 234ms
2496:	learn: 0.1797046	total: 2m 26s	remaining: 176ms
2497:	learn: 0.1796577	total: 2m 26s	remaining: 117ms
2498:	learn: 0.1796074	total: 2m 26s	remaining: 58.6ms
2499:	learn: 0.1795484	total: 2m 26s	remaining: 0us


In [425]:
print('Accuracy of catboost: ', cbc.score(X_test, y_test)) ## using default threshold 0.5 of catboost 
reporting(cbc.predict_proba(X_test)[:, 1], y_test)

Accuracy of catboost:  0.86075

AUC score:       0.8120
Best threshold       0.1700
Report with best threshold : 


Confusion Matrix : 
 [[2704  692]
 [ 216  388]]

Accuracy    :       0.7730
Sensitivity :       0.9260
Specificity :       0.3593


In [420]:
private_test_cb = ft_engineering_test(test_data, remove_cols, use_catboost = True)
pred = cbc.predict_proba(private_test_cb)[:, 1]

(10000, 95)


## C. Using Logistic Regression 

### 1. Train and test 
- There is a potential leakage in data since when we perform one-hot encoder, we expose all the data. I tried to avoid the data leakage, but the exercise_40_train.csv couldn't be splitted stratifily on all categorical features. 
- In order to minimize the leakage, I fill NaN with 0 in numerical features, and replace the missing values in categorical features with the most common values. Doing in so will not leak any information in numerical features, and only expose the most common values in categorical features.
- The valinally logistic regression doesn't give good accuracy, I added a feature selection method RFECv which is a feature ranking with recursive feature elimination and cross-validated selection of the best number of features.

In [413]:
### clean and extract features
data_lr = ft_engineering_train(train_data, remove_cols, use_catboost = False)

### split train and test set 
X_train, X_test, y_train, y_test = train_test_split(
                                    data_lr.loc[:,~data_lr.columns.isin(['y'])],
                                    data_lr['y'],
                                    test_size=0.1,
                                    random_state=0)

In [414]:
clf = RFECV(LogisticRegression(random_state=0, C = 0.2),
      scoring = 'roc_auc',
      cv = 5,
      step = 5)
clf.fit(X_train, y_train)

RFECV(cv=5, estimator=LogisticRegression(C=0.2, random_state=0),
      scoring='roc_auc', step=5)

In [416]:
#Generate predicted probabilites
clf_probs = clf.predict_proba(X_test)
print('Accuracy: ', clf.score(X_test, y_test))
reporting(clf_probs[:,1], y_test)

Accuracy:  0.849

AUC score:       0.6980
Best threshold       0.1400
Report with best threshold : 


Confusion Matrix : 
 [[1953 1443]
 [ 173  431]]

Accuracy    :       0.5960
Sensitivity :       0.9186
Specificity :       0.2300


#### Export the probability for test data for submission 

In [366]:
private_test_lr = ft_engineering_test(test_data, remove_cols, use_catboost = False)
clf.predict_proba(private_test_lr)[:,1]

array([0.13311086, 0.13446383, 0.13954559, ..., 0.1411462 , 0.13372441,
       0.13384741])

### Removing features using lasso
- This is just an extra step (Feel free to ignore)

In [303]:
sel_ = SelectFromModel(LogisticRegression(penalty='l1', solver='saga', C = 0.1))
sel_.fit(scaler.transform(X_train), y_train)

SelectFromModel(estimator=LogisticRegression(C=0.1, penalty='l1',
                                             solver='saga'))

In [304]:
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 175
selected features: 149
features with coefficients shrank to zero: 26


In [305]:
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index([ 'x5',  'x9', 'x11', 'x26', 'x35', 'x48', 'x90', 'x96',     1,     4,
           7,     8,    17,    22,    35,    43,    50,    52,    60,    63,
          74,    77,    79,    80,    81,    84],
      dtype='object')

In [306]:
X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_test)

In [307]:
clf_selected = LogisticRegression(random_state=0, C = 0.2)
clf_selected.fit(X_train_selected, y_train)

In [319]:
#Generate predicted probabilites
clf_probs = clf_selected.predict_proba(X_test_selected)
print('AUC: ', roc_auc_score(y_test, clf_probs[:,1]))
print('Accuracy: ', clf_selected.score(X_test_selected, y_test))

AUC:  0.5858263928560691
Accuracy:  0.842


In [320]:
reporting(clf_selected.predict_proba(X_test_selected)[:,1], y_test)


AUC score:       0.5858
Best threshold       0.1300

Confusion Matrix : 
 [[853 831]
 [120 196]]

Accuracy    :       0.5245
Sensitivity :       0.8767
Specificity :       0.1908
